In [1]:
import os
import numpy as np
import pandas as pd
from pandasql import PandaSQL
pdsql = PandaSQL()

In [2]:
os.chdir("/Users/JamesWang1/Documents/ML/Duke Hack-a-thon_2018MBB")

In [3]:
df_duke = pd.read_csv("Duke Data_2018.csv",dtype={'min': np.float64})

/Users/JamesWang1/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (18,19,54,63,71,90,91,92) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_duke.head()

,id,playerid,gender,sport,sport2,sport3,sport4,pre,firstshort,checkname,...,fga3_pct,tp3,treb_g,ast_g,stl_g,blk_g,tp_g,ft_pct,ast_to,differential
0,258,258,m,mbb,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,...,0.5,6,5.0,0.0,0.0,1.0,6,0.0,0.0,17
1,2748,2748,m,mbb,<Null>,<Null>,<Null>,<Null>,<Null>,"PLUMLEE,MARSHALL",...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,-12
2,2753,2753,m,mbb,<Null>,<Null>,<Null>,<Null>,<Null>,"JONES,MATT",...,0.0,6,2.0,1.0,3.0,0.0,6,1.0,0.5,7
3,2740,2740,m,mbb,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,...,0.0,2,4.0,1.0,0.0,1.0,2,0.0,0.0,4
4,495,495,m,mbb,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,-13


In [12]:
# check missing data 
df_duke.columns[(df_duke.isnull().sum(axis = 0)/df_duke.shape[0])>.2]

Index(['opprecord', 'oppcoachid', 'oppcoach', 'code', 'tiptime', 'tv', 'tv2',
       'gamenoconf', 'gamenoacc', 'gamenohome', 'gamenocameron', 'gamenoaway',
       'gamenoneu', 'gamenops', 'gamenoact', 'gamenosct', 'gamenoct',
       'gamenoncaa', 'gamenonit', 'gamenoconfsea', 'gamenohomesea',
       'gamenoawaysea', 'gamenoneusea', 'starttime', 'endtime', 'duration'],
      dtype='object')

In [6]:
df_duke.select_dtypes(include = ['object']).columns

Index(['gender', 'sport', 'sport2', 'sport3', 'sport4', 'pre', 'firstshort',
       'checkname', 'first', 'middle', 'last', 'post', 'maiden', 'nickname',
       'nickname2', 'nickname3', 'bmonth', 'bday', 'byear', 'birthdate',
       'hometown', 'homestate', 'homecountry', 'birthtown', 'birthstate',
       'birthcountry', 'highschool', 'highschool2', 'college', 'prevschool',
       'major1', 'major2', 'minor', 'graduated', 'active', 'opponent2', 'date',
       'season_game_info', 'loc_game_info', 'outcome', 'opprecord',
       'headcoach', 'oppcoach', 'city', 'state', 'code', 'venue', 'tiptime',
       'tv', 'tv2', 'starttime', 'endtime', 'duration', 'game_date',
       'month_name', 'day_name', 'month_name_full', 'CONF_Y_N', 'name',
       'opponent', 'gspos', 'season', 'loc', 'GS_Y_N', 'fga3_flag',
       'postseason_flag', 'acc_flag', 'ncaa_flag', 'fg3_flag', 'GS_Y_N_NEW',
       'CONF_Y_N_NEW'],
      dtype='object')

In [7]:
#df_duke.loc[df_duke['first'] == "Grayson", :]

In [8]:
print(list(df_duke.columns))

['id', 'playerid', 'gender', 'sport', 'sport2', 'sport3', 'sport4', 'pre', 'firstshort', 'checkname', 'first', 'middle', 'last', 'post', 'maiden', 'nickname', 'nickname2', 'nickname3', 'bmonth', 'bday', 'byear', 'birthdate', 'hometown', 'homestate', 'homecountry', 'birthtown', 'birthstate', 'birthcountry', 'highschool', 'highschool2', 'college', 'prevschool', 'major1', 'major2', 'minor', 'graduated', 'active', 'AT_GAMEINFO_id', 'gameid_game_info', 'opponent_game_info', 'opponent2', 'month', 'day', 'year', 'date', 'season_game_info', 'season4', 'season2', 'loc_game_info', 'outcome', 'wins', 'losses', 'accwins', 'acclosses', 'opprecord', 'dukescore', 'oppscore', 'ot', 'dukerank', 'opprank', 'coachid', 'headcoach', 'oppcoachid', 'oppcoach', 'city', 'state', 'code', 'venue', 'attendance', 'tiptime', 'tv', 'tv2', 'gameno', 'gamenoconf', 'gamenoacc', 'gamenohome', 'gamenocameron', 'gamenoaway', 'gamenoneu', 'gamenops', 'gamenoact', 'gamenosct', 'gamenoct', 'gamenoncaa', 'gamenonit', 'gamenos

In [9]:
df1 = df_duke.groupby(['playerid', 'first','last','season4'])['tp','treb','min'].mean()

In [112]:
df_duke.loc[df_duke['last']=='Irving'].shape[0] # Irving only played 11 games

11

Best season of players who at least play 10 min and 5 points

In [113]:
eachSeason = pdsql("""
SELECT playerid, first, last, season4, gspos, AVG(tp) as ppg, AVG(min) as mpg , 
    AVG(treb) as rpg, AVG(ast) as apg, AVG(blk) as bpg, AVG(stl) as spg, SUM(ft)/SUM(fga) as gf, 
    SUM(fg3)/SUM(fga3) as dgf3, AVG(ast_to) AS to_ast,  AVG(fga3) as fga3, count(*) as gamePlayed
FROM df_duke
GROUP BY playerid, season4
HAVING AVG(min) > 25 AND AVG(tp) >10 AND count(*) > 10 AND SUM(fga) > 0 AND SUM(fga3)> 2 
""")
eachSeason.tail(5)
# how to make it their last season? or their best season

,playerid,first,last,season4,gspos,ppg,mpg,rpg,apg,bpg,spg,gf,dgf3,to_ast,fga3,gamePlayed
94,2774,Jayson,Tatum,2017,f,16.827586,33.310345,7.344828,2.137931,1.137931,1.344828,0.323288,0.341880,0.944483,4.034483,29
95,2776,Marvin,Bagley III,2018,f,21.545455,32.636364,11.409091,1.545455,1.090909,1.000000,0.312500,0.340909,0.737727,2.000000,22
96,2778,Wendell,Carter,2018,f,14.454545,26.409091,9.318182,1.909091,2.000000,0.772727,0.347150,0.464286,0.794545,1.272727,22
97,2779,Trevon,Duval,2018,g,11.272727,29.590909,2.045455,6.000000,0.181818,1.590909,0.184834,0.267857,2.934091,2.545455,22
98,2782,Gary,Trent,2018,g,14.727273,32.590909,4.363636,1.681818,0.136364,1.090909,0.200803,0.434783,0.882273,6.272727,22


In [114]:
eachSeason.shape

(99, 16)

In [115]:
eachSeason.loc[eachSeason['last'] =='Irving']

,playerid,first,last,season4,gspos,ppg,mpg,rpg,apg,bpg,spg,gf,dgf3,to_ast,fga3,gamePlayed
77,2742,Kyrie,Irving,2011,<Null>,17.454545,27.545455,3.363636,4.272727,0.545455,1.454545,0.615385,0.461538,1.844545,3.545455,11


In [118]:
bestSeason = pdsql("""
SELECT playerid, first, last, season4, gspos, max(mpg) as mpg, ppg, rpg, apg, bpg, spg, gf, dgf3, to_ast, fga3
FROM eachSeason
GROUP BY playerid
""")
bestSeason.head(2)

,playerid,first,last,season4,gspos,mpg,ppg,rpg,apg,bpg,spg,gf,dgf3,to_ast,fga3
0,8,Tommy,Amaker,1987,<Null>,35.606061,12.303030,2.333333,3.636364,0.090909,1.757576,0.143678,0.427184,2.111818,3.121212
1,14,William,Avery,1999,<Null>,31.025641,14.871795,3.512821,5.025641,0.025641,1.461538,0.245192,0.410811,2.145385,4.743590


compute stats of players for avg of their best season (stats per 40 min), make radar chart

In [120]:
# stats per 40 min
df2 = bestSeason.loc[:,'ppg':'spg'].apply(lambda x: x/bestSeason['mpg'] * 40).apply(lambda x: (x-x.mean())/x.std() + 3)
df2stat = bestSeason.loc[:,'gf':'fga3'].apply(lambda x: (x-x.mean())/x.std() + 3)
# rename column to col40

print(df2.columns)
df2['z'] = df2['ppg']
df2 = df2.join(df2stat)
df2.rename(columns = lambda col : "".join([col,"1"]), inplace = True)
print(df2.columns)
df22= bestSeason.join(df2)
# tall table
df3 = pd.melt(df22, id_vars = ['playerid'], var_name = 'measure', value_vars=['ppg1', 'rpg1','apg1','bpg1','spg1','z1', 'gf1','dgf31','to_ast1'], value_name='stats')

Index(['ppg', 'rpg', 'apg', 'bpg', 'spg'], dtype='object')
Index(['ppg1', 'rpg1', 'apg1', 'bpg1', 'spg1', 'z1', 'gf1', 'dgf31', 'to_ast1',
       'fga31'],
      dtype='object')


In [121]:
df2.describe()

,ppg1,rpg1,apg1,bpg1,spg1,z1,gf1,dgf31,to_ast1,fga31
count,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000
mean,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
std,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
min,1.057013,1.712610,1.728185,2.150399,1.284723,1.057013,1.400405,-0.170559,1.528038,0.943200
25%,2.393480,2.308829,2.398608,2.306674,2.178838,2.393480,2.228386,2.496734,2.242342,2.406576
50%,2.970194,2.796399,2.619412,2.554853,2.847320,2.970194,2.987189,3.094441,2.713410,2.750899
75%,3.660335,3.533697,3.404544,3.428376,3.639801,3.660335,3.496655,3.584820,3.787360,3.662892
max,5.216550,5.728221,6.148279,7.117309,5.528351,5.216550,6.465065,5.944031,5.635900,5.552442


Make Radar Coordinates

In [122]:
c1 = np.cos(2/5 *np.pi)
c2 = np.cos(1/5 * np.pi)
s1 = np.sin(2/5 * np.pi)
s2 = np.sin(1/5 * np.pi)
#radar_coord = dict({"p": (0,1), 'r': (s1,c1), 's':(s2,-c2), 'b': (-s2,-c2), 'a' : (-s1,c1)})
radar_coord = dict({"p": [0,1],"z": [0,1], 'r': [s1,c1], 's':[s2,-c2], 'b': [-s2,-c2], 'a' : [-s1,c1],
                   'g': [-s2,-c2],'d': [-s2,-c2],'t': [-s2,-c2]})

In [123]:
# change label to just p,b, s, etc
df3['measure'] =df3['measure'].apply(lambda x : x[0])

# normalize stats
#df3['stats_norm'] = (df3['stats']-df3['stats'].mean())/df3['stats'].std() + 1

#df3['stats_x'] = df3['stats_norm'] * radar_coord[df3['measure']][0]
df3['stats_x'] = [radar_coord[x][0] for x in df3['measure']] * df3['stats'] 
df3['stats_y'] = [radar_coord[x][1] for x in df3['measure']] * df3['stats'] 
df3.groupby('measure')['stats','stats'].mean() # seems right
df3.head(2)

,playerid,measure,stats,stats_x,stats_y
0,8,p,1.285186,0.0,1.285186
1,14,p,2.860067,0.0,2.860067


In [124]:
bestSeason.head(1)

,playerid,first,last,season4,gspos,mpg,ppg,rpg,apg,bpg,spg,gf,dgf3,to_ast,fga3
0,8,Tommy,Amaker,1987,<Null>,35.606061,12.30303,2.333333,3.636364,0.090909,1.757576,0.143678,0.427184,2.111818,3.121212


In [125]:
# use merge not join. join is troublesome with overlapping columns
df_join = bestSeason.merge(df3, on = 'playerid',how = 'inner')

df_join['ID1'] = 10 
df_join['ID1'][df_join['measure'] == 'p']= 2 
df_join['ID1'][df_join['measure'] == 'r']= 4 
df_join['ID1'][df_join['measure'] == 's']= 6 
df_join['ID1'][df_join['measure'] == 'b']= 8 
df_join['ID1'][df_join['measure'] == 't']= 8 
df_join['ID1'][df_join['measure'] == 'g']= 8 
df_join['ID1'][df_join['measure'] == 'd']= 8 
df_join['ID1'][df_join['measure'] == 'z']= 11
#firstlast
df_join['playerName'] = df_join['first'] + " " + df_join['last']

df_join.head(2)

/Users/JamesWang1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/JamesWang1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/JamesWang1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/JamesWang1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithC

,playerid,first,last,season4,gspos,mpg,ppg,rpg,apg,bpg,...,gf,dgf3,to_ast,fga3,measure,stats,stats_x,stats_y,ID1,playerName
0,8,Tommy,Amaker,1987,<Null>,35.606061,12.30303,2.333333,3.636364,0.090909,...,0.143678,0.427184,2.111818,3.121212,p,1.285186,0.000000,1.285186,2,Tommy Amaker
1,8,Tommy,Amaker,1987,<Null>,35.606061,12.30303,2.333333,3.636364,0.090909,...,0.143678,0.427184,2.111818,3.121212,r,1.712610,1.628789,0.529225,4,Tommy Amaker


In [129]:
df_join.to_csv('MBB_radar9.csv')

In [128]:
df_join.loc[df_join['last'] =='Irving']

,playerid,first,last,season4,gspos,mpg,ppg,rpg,apg,bpg,...,gf,dgf3,to_ast,fga3,measure,stats,stats_x,stats_y,ID1,playerName
324,2742,Kyrie,Irving,2011,<Null>,27.545455,17.454545,3.363636,4.272727,0.545455,...,0.615385,0.461538,1.844545,3.545455,p,4.676435,0.000000,4.676435,2,Kyrie Irving
325,2742,Kyrie,Irving,2011,<Null>,27.545455,17.454545,3.363636,4.272727,0.545455,...,0.615385,0.461538,1.844545,3.545455,r,2.503414,2.380888,0.773598,4,Kyrie Irving
326,2742,Kyrie,Irving,2011,<Null>,27.545455,17.454545,3.363636,4.272727,0.545455,...,0.615385,0.461538,1.844545,3.545455,a,4.501658,-4.281331,1.391089,10,Kyrie Irving
327,2742,Kyrie,Irving,2011,<Null>,27.545455,17.454545,3.363636,4.272727,0.545455,...,0.615385,0.461538,1.844545,3.545455,b,2.984576,-1.754290,-2.414573,8,Kyrie Irving
328,2742,Kyrie,Irving,2011,<Null>,27.545455,17.454545,3.363636,4.272727,0.545455,...,0.615385,0.461538,1.844545,3.545455,s,4.090087,2.404093,-3.308950,6,Kyrie Irving
329,2742,Kyrie,Irving,2011,<Null>,27.545455,17.454545,3.363636,4.272727,0.545455,...,0.615385,0.461538,1.844545,3.545455,z,4.676435,0.000000,4.676435,11,Kyrie Irving
330,2742,Kyrie,Irving,2011,<Null>,27.545455,17.454545,3.363636,4.272727,0.545455,...,0.615385,0.461538,1.844545,3.545455,g,6.465065,-3.800070,-5.230348,8,Kyrie Irving
331,2742,Kyrie,Irving,2011,<Null>,27.545455,17.454545,3.363636,4.272727,0.545455,...,0.615385,0.461538,1.844545,3.545455,d,4.288954,-2.520984,-3.469837,8,Kyrie Irving
332,2742,Kyrie,Irving,2011,<Null>,27.545455,17.454545,3.363636,4.272727,0.545455,...,0.615385,0.461538,1.844545,3.545455,t,3.787360,-2.226155,-3.064039,8,Kyrie Irving
